In [45]:
## 分類をインデックス化してDataFrameに追加する共通関数(秋山さん作成分)
def classify_and_add_indexed_df(df, org_key, new_key):
    class_sets = set()
    for class_ in df[org_key]:
        class_sets.add(class_)

    sorted_classes = sorted(list(class_sets))
    class_indices = dict((c, i) for i, c in enumerate(sorted_classes)) 
#    print(class_indices)

    classes = []
    for i, class_ in enumerate(reviews[org_key]):
        classes.append(class_indices[class_])
    indexed_df[new_key] = classes

In [42]:
# 単語分割(See. http://www.denzow.me/entry/2017/10/29/160903)
from janome.tokenizer import Tokenizer

def split_into_words(doc):
    """
    名詞だけを取り出してリストで戻す関数
    """
    try:
        t = Tokenizer(mmap=True)
        word_list = []
        # 形態素して取り出す
        for token in t.tokenize(doc):
            # 品詞の判定をして、名詞か動詞か形容詞だけを取り出す
            if (token.part_of_speech.split(",")[0] in ("名詞","動詞","形容詞")
                and  token.part_of_speech.split(",")[1] != "数"):  # ただし、数詞は使っても意味が薄いので捨てる
                # 表層形を登録する
                word_list.append(token.surface)
        return word_list
    except Exception as ex:
        print(ex)
        return []

In [121]:
## メイン関数
import sys
import codecs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

## ファイルの読込み(全件)
#shops = pd.read_csv('shops.txt', delimiter='\t')
#reviews = pd.read_csv('reviews.txt', delimiter='\t')

## ファイルの読込み(サンプル)
shops = pd.read_csv('as.txt', delimiter='\t')
reviews = pd.read_csv('ar.txt', delimiter='\t')

## ファイルの先頭と末端読み(学習データと検証データ分割用)
#reviewsh = reviews.head(3)
#reviewst = reviews.tail(3)
#print(reviewsh)
#print(reviewst)

##学習用データ前作業
### インデックスの作成
indexed_df = pd.DataFrame()
classify_and_add_indexed_df(reviews, '分類', 'category')
classify_and_add_indexed_df(reviews, 'スープ', 'soup')

### 文章の形態素解析
for i, doc1 in enumerate(reviews['メニュー']):
    words1 = split_into_words(doc1)
    indexed_df.loc[i, 'menu'] = i

for i, doc2 in enumerate(reviews['コメント']):
    words2 = split_into_words(doc2.replace('<br />', ''))
    indexed_df.loc[i, 'comment'] = i

print(indexed_df)

### 学習データとサンプルデータとの分割
train_X, test_X, train_Y, test_Y = train_test_split(indexed_df, reviews['点数'], train_size=0.8)
print(np.array(train_X))
#print(test_X)
print(np.array(train_Y))
#print(test_Y)

## モデルの構築
"""多層パーセプトロンモデルを構築()"""
model = Sequential()
model.add(Dense(3, input_dim=4))    # 入力層4ノード, 出力層100
model.add(Activation('sigmoid'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# model.add(Dense(100)) # 出力層100ノード(0～100点),全結合
#model.add(Activation('softmax'))
model.compile(optimizer=SGD(lr=0.1), loss='binary_crossentropy', metrics=['accuracy'])

# モデル訓練
model.fit(np.array(train_X), np.array(train_Y), epochs=5, batch_size=1)

# モデル評価
loss_and_metrics = model.evaluate(test_X, test_Y)

   category  soup  menu  comment
0         1     1   0.0      0.0
1         1     0   1.0      1.0
2         1     2   2.0      2.0
3         0     2   3.0      3.0
4         1     2   4.0      4.0
[[ 0.  2.  3.  3.]
 [ 1.  2.  4.  4.]
 [ 1.  0.  1.  1.]
 [ 1.  2.  2.  2.]]
[87 80 70 83]
Epoch 1/5
4/4 [==============================] - 0s - loss: -903.5170 - acc: 0.0000e+00   
Epoch 2/5
4/4 [==============================] - 0s - loss: -1259.4484 - acc: 0.0000e+00     
Epoch 3/5
4/4 [==============================] - 0s - loss: -1259.4484 - acc: 0.0000e+00     
Epoch 4/5
4/4 [==============================] - 0s - loss: -1259.4484 - acc: 0.0000e+00     
Epoch 5/5

ValueError: Cannot feed value of shape (1, 1) for Tensor 'dense_62_input:0', which has shape '(?, 4)'

## 参考URL
### LSTMでどのキャラクターのセリフか判別する
### https://qiita.com/CookieBox26/items/6823346661f600b246eb
### 式を図示化してくれているので分かりやすかった

In [ ]:
#print(shops.columns[1])
#print(shops['店名'])
#print(reviews['コメント'])